In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os

from CN_utils import *

In [3]:
# Directory containing the .tsv FACETS CN segments files for each dataset, including filtered ones.
# Default repo location given.
# External datasets not included with our repo- you will have to generate these yourself

input_dir = "../../data/copy_number/"
dataset_names = ["HTAN_WGS", "HTAN_WES", "PUTH", "SCORT"]

resource_dir = "../../data/resource/"

save_dir = input_dir+"fraction_genome_altered/"

In [4]:
removed_regions_loc = resource_dir+"hg38.UCSC.centromere.telomere.encode.bed"

removed_regions = pd.read_csv(removed_regions_loc, sep="\t", names=["chrom", "start_pos", "end_pos", "length", "source", "reason"])

telomere_table = removed_regions[removed_regions["reason"]=="telomere"]

telomere_dict = {}
for chrom in list(set(telomere_table["chrom"])):
    only_chrom = telomere_table[telomere_table["chrom"]==chrom]
    telomere_dict[chrom] = [only_chrom.iloc[0]["end_pos"], only_chrom.iloc[1]["start_pos"]]
    
genome_len = sum([telomere_dict[x][1]-telomere_dict[x][0] if not x in ["chrX", "chrY"] else 0 for x in telomere_dict])

In [5]:
merged_filtered_datasets = []
WGD_info = []
for dataset in dataset_names:
    data_to_add = pd.read_csv(input_dir+dataset+"_CN_filtered_merged.tsv", sep="\t")
    merged_filtered_datasets.append(data_to_add)
    doubling_to_add = pd.read_csv(input_dir+"genome_doubling/"+dataset+"_doubled.tsv", sep="\t", index_col=0)
    WGD_info.append(dict(zip(doubling_to_add.index, doubling_to_add["genome_doubled"])))

In [6]:
for i,to_fga in enumerate(merged_filtered_datasets):
    save_name = dataset_names[i]
    fga = fraction_genome_altered(to_fga, genome_len, WGD_info[i]).sort_index()
    fga.to_csv(save_dir+save_name+"_fga.tsv", sep="\t", header=False)

/oak/stanford/groups/ccurtis2/users/debra/FAP/py_notebooks/final_for_submission/analysis/copy_number/CN_utils.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_normal_cn["length"] = not_normal_cn["loc_end"] - not_normal_cn["loc_start"]
